<a href="https://githubtocolab.com/Eunseob/purdue_me597/blob/main/lab/lab5/L5_Colab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 5.1 Middleware 2 - MTConnect Adapter

## Learning Goals

Students will be able to:
1. Combine multiple sensors into a single MTConnect Adapter
2. Configure multiple adapters for each sensor
3. Modify and configure code for the MTConnect adapters
4. Utilize HTTP Requests and XPath to extract information

## 1.1 Introduction

In the last lab (Lab5), we ran MTConnect agent in various configurations by simulating an MTConnect adapter based on log data file. In Prelab 5, we programmed an MTConnect adapter using random value generations. In this lab, we will program the MTConnect adapters to collect sensors data connected to Raspberry Pi. To be specific, the target sensors are humidity-temperature sensor (DHT11, Lab1), accelerometer (ADXL345, Lab2), power meter (Acuvim-CL, Lab3), and IO-Link (IFM AL1302, Lab3, **OPTIONAL**). In addition, we will practice more about basic client application using XPath and XML parsing. 

## 1.2 A Singular MTConnect Adapter for Multiple Sensors


First, let’s make sensor connections to Raspberry Pi and check if all sensors are working. By using jumper wires on the sensors, make connections as Table 1. If you are not sure, please take look at manuals of Lab 2 (IoT sensor data collection 1 – Humidity and temperature) and Lab 3 (IoT sensor data collection 2 – Accelerometer and signal processing) for the details of each pin information and connection. The wiring diagram and configuration are shown in Figure 1 and Figure 2. Because power meter will be connected through the network, you do not need to make physical connections to the power meter. However, please make sure that your Raspberry Pi and laptop must be on the router network.

*Table 1 Pin wiring connections for DHT11 and ADXL345*

<table width="70%">
<thead>
  <tr>
    <th>Sensor</th>
    <th>Pin</th>
    <th>Raspberry Pi GPIO</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td rowspan="3">DHT11<br>(Temperature and humidity)</td>
    <td>Pin 1 (V<sub>CC</sub>, +)</td>
    <td>Pin 17 (3.3V power*)</td>
  </tr>
  <tr>
    <td>Pin 2 (Data, out)</td>
    <td>Pin 16 (GPIO 23)</td>
  </tr>
  <tr>
    <td>Pin 4 (GND, -)</td>
    <td>Pin 14 (GND*)</td>
  </tr>
  <tr>
    <td rowspan="4">ADXL345<br>(3-axis accelerometer)</td>
    <td>Vcc (of V<sub>IN</sub>)</td>
    <td>Pin 1 (3.3V power*)</td>
  </tr>
  <tr>
    <td>GND</td>
    <td>Pin 6 (GND*)</td>
  </tr>
  <tr>
    <td>SDA</td>
    <td>Pin 3 (SDA)</td>
  </tr>
  <tr>
    <td>SCL</td>
    <td>Pin 5 (SCL)</td>
  </tr>
</tbody>
</table>

<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab5_fig1.png?raw=true" width="70%">

*Figure 1 Wiring diagram for DHT11 and ADXL345 to Raspberry Pi*

<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab5_fig2.png?raw=true" width="60%">

*Figure 2 Hardware configuration for wiring of DHT11 and ADXL345 to Raspberry Pi*

Next, check each sensor if it is working. You can reuse the sample codes from previous labs, but the sample 
Python codes, [lab5_ADXL345_sample.py](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab5/lab5_sample/lab5_ADXL345_sample.py), [lab5_DHT11_sample.py](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab5/lab5_sample/lab5_DHT11_sample.py), and [lab5_powermeter_sample.py](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab5/lab5_sample/lab5_powermeter_sample.py), are on [Github repo](https://github.com/Eunseob/purdue_me597/tree/main/lab/lab5/lab5_sample). Please read the comments on the sample codes if you are hard to understand the program. All sample codes are running infinitely in a while loop if not being forcedly halted. Please note that to stop each Python code, press Ctrl + c. 






### Task 1.1

Take a picture of your wiring configuration as Figure 2 and attach it to the report. 

---

Place your picture here.

---




### Task 1.2

Run each sample code to check each sensor using ‘Terminal’ on Raspberry Pi and then capture each ‘Terminal’ while running. Attach those to the report. 

Note that you can run all code at the same time. Open multiple terminals and run one code per terminal. Command to run Python3 in Raspberry Pi file is below.

  * *If you get access to Raspberry Pi via SSH, try to use [tmux](https://github.com/tmux/tmux/wiki) to have multiple terminals in single window.*

---

**<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/icon_RaspberryPi.png?raw=tru" width="20">Raspberry Pi - Terminal**

`python3 code_file_name.py`

  * *The program (code_file_name.py) must be in your terminal working directory*

---

---

Place your screenshot for ADXL345 sensor here.

---

---

Place your screenshot for DHT11 sensor here.

---

---

Place your screenshot for Power meter here.

---

### 1.3 Single MTConnect adapter for all sensors

Let’s start to program the simplest, single MTConnect adapter with the same sampling period for all sensors. The schematic of the MTConnect configuration of single MTConnect adapter for all sensors is shown in Figure 3.  

<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab5_fig3.png?raw=true" width="100%">

*Figure 3 MTConnect configuration 1: Single adapter case*

DHT11 and ADXL345 sensors are connected to Raspberry Pi using jumper wires. The power meter measuring electricity of the vacuum pump is hooked up to the router. You can get power meter data via Modbus TCP as Lab 3. A single MTConnect adapter is supposed to collect all sensor data and transmit to MTConnect agent. Both are on Raspberry Pi. To make it simple, the sampling period for all sensors is 2 seconds. The descriptions about data items are as follows. 

* ADXL345 sensor
  * X-axis acceleration (Data category=SAMPLE, unit=mm/s<sup>2</sup>)
  * Y-axis acceleration (Data category=SAMPLE, unit=mm/s<sup>2</sup>)
  * Z-axis acceleration (Data type=SAMPLE, unit=mm/s<sup>2</sup>) 
* DHT11 sensor
  * Temperature (Data category=SAMPLE, unit=°C)
  * Humidity (Data category=SAMPLE, unit=RH%) 
* Power meter
  * True power (Data category=SAMPLE, unit=W)
  * Power state (Data category=EVENT) 

By measuring the power consumption of a machine, we can tell whether the machine is turned on or not. In other words, say, the power state is ‘ON’ or ‘OFF’. This is an example of *'EVENT'* data item. **The logic to determine the power state event in this example is that if measure true power consumption is bigger than 30 W, the power state is ‘ON’. If not, the power state is ‘OFF’.** Because the Modbuc TCP/RTU gateway and power meter itself are plugged in the outlet for measuring electrical power, idle power consumption is around 20 W. Also, the vacuum pump consumes more than 100 W in any states. 

First, modify the MTConnect agent configurations on Raspberry Pi. Please use the agent directory we used the previous lab. Or you can create new directory by copying all MTConnect agent files in it. Please follow the step below. 

1.	Modify *‘agent.cfg’* as Figure 4. 

  a.	You do not need to modify other lines. 

  b.	Please note that MTConnect agent port is 5000 and MTConnect adapter port is 7878. 

  c.	Host IP of both is 127.0.0.1 or localhost.

  <img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab5_fig4.png?raw=true" width="60%">

   *Figure 4 'agent.cfg' for Part 2*
 
2.	Modify ‘Device.xml’ as below. 

---

**<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/icon_Tip.png?raw=tru" width="20">Tip - Comment out in XML document**

You may want to leave comments on XML document or to make comment out lines for any reasons. To do that, you can use ‘\<!-- 'ANY COMMENT’ -->’. Any information between ‘\<!--' and ‘-->’ does not affect the XML document as line 15 of Figure 5. 

Please note that this configuration of MTConnect device is the simplest way. When you build MTConnect device structure, please take a look at MTConnect standard Part 2.0 – Devices Information Model 

---

  <img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab5_fig5.png?raw=true" width="100%">

*Figure 5 'Device.xml' for Part 2*

3.	Run MTConnect agent in ‘Terminal’ 

---

**<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/icon_RaspberryPi.png?raw=tru" width="20">Raspberry Pi - Terminal**

`sudo ./agent`

<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab5_img1.png?raw=true" width="50%">

---

4.	Check if the agent works by HTTP request as Figure 6 from a web browser of laptop. 

  a.	Please note that you need to use Raspberry Pi’s IP address on the router network. 

  b.	Presumably if you request ‘current’ from the agent, all values must be ‘UNAVAILABLE’ as Figure 7 because no adapter connection was made. 

  <img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab5_fig6.png?raw=true" width="100%">

  *Figure 6 'probe' request to MTConnect agent for Part 2*

  <img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab5_fig7.png?raw=true" width="90%">

  *Figure 7 ‘current’ response of MTConnect agent without adapter connection*

Now, you are ready to run MTConnect adapter. Please download [lab5_adapter.zip](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab5/lab5_adapter.zip?raw=true) from the Github repo and then extract the zip file on Raspberry Pi. The descriptions about each file in 'lab5_adapter.zip’ are below. 

* [data_item.py](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab5/lab5_adapter/data_item.py?raw=true)
  * Data item package for MTConnect (by System Insights©) 
* [mtconnect_adapter.py](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab5/lab5_adapter/mtconnect_adapter.py?raw=true)
  * MTConnect adapter package (by System Insights©) 
* [lab5_adapter1.py](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab5/lab5_adapter/lab5_adapter1.py?raw=true)
  * Sample adapter program for Part 2 (**incomplete**) 
  * Collecting and transmitting all sensors, DHT11, ADXL345, and power meter, to MTConnect agent. 
* [lab5_adapter2-1.py](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab5/lab5_adapter/lab5_adapter2-1.py?raw=true)
  * Sample adapter program for Part 3 (**incomplete**)
  * Only DHT11 
* [lab5_adapter2-2.py](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab5/lab5_adapter/lab5_adapter2-2.py?raw=true)
  * Sample adapter program for Part 3 (**incomplete**)
  * Only ADXL345 
* [lab5_adapter2-3.py]((https://github.com/Eunseob/purdue_me597/blob/main/lab/lab5/lab5_adapter/lab5_adapter2-3.py?raw=true))
  * Sample adapter program for Part 3 (**incomplete**)
  * Only power meter 

Please take a look at comments in each line of sample programs. Run ‘lab5_adapter1.py’ first on Raspberry Pi. And then, check the MTConnect agent using HTTP ‘current’ request of the web browser on laptop. As you can see, some values, Y-axis acceleration, Z-axis acceleration, and humidity, are ‘UNAVAILABLE’ because the sample code is incomplete. 

<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab5_fig8.png?raw=true" width="90%">

*Figure 8 ‘current’ response of MTConnect agent without modification of ‘lab6_adapter1.py’*





#### Task 1.3

3.	Save the XML document after changing to ‘View page source’ as ‘Lab5_TASK1.3.xml’ and then upload it on Brightspace. 


  1. Modify ‘lab6_adapter1.py’ and make it complete to have all values available and then run the adapter.
  2. Capture a web browser as Figure 9 after requesting ‘current’ from the agent and attach it to the report.

  ---

  Place your screenshot here.

  ---

![picture](https://github.com/hewp84/tinyml/blob/main/img/L6_Figure9.png?raw=true)

*Figure 9 ‘current’ response of MTConnect agent after modification of ‘lab6_adapter1.py’*

In this case (a single MTConnect adapter for multiple sensors), it is hard to make different sampling periods of each sensor. Of course, you can but it requires a high level of programming. If you are interested, study multiprocessing and multithreading. 

<br></br>

Please continue to [Lab 5.2 here](L6_Colab2.ipynb).
